### To run this model yourself: go to Run -> Run All Cells in the top left menu bar.

In [ ]:
from epx import Job, ModelConfig, SynthPop

import pandas as pd
import json
import plotly.express as px
import plotly.io as pio
import time
import plotly.graph_objects as go
import requests

# Use the Epistemix default plotly template
r = requests.get("https://gist.githubusercontent.com/daniel-epistemix/8009ad31ebfa96ac97b7be038c014c0d/raw/320c3b0ca3dfbf7946e49c97254fa65d4753aeac/epx_plotly_theme.json")
if r.status_code == 200:
    pio.templates["epistemix"] = go.layout.Template(r.json())
    pio.templates.default = "epistemix"

# Schelling Model

An implementation of the Schelling model of racial segregation in housing, using the household locations (N~8000) of Kewaunee County, Wisconsin. If you'd like to learn more about how this model works in detail, check out Lesson 8 in the Quickstart Guide.

In brief, agents are assigned one of two colors, in this case blue and red. At each timestep in the simulation, agents assess the color distribution of the residents in their block group and determine if enough agents are of a similar color to themselves.

If the agent is happy with the number of similar agents, they remain in their current household. If they are unhappy, the seek a new location to live from a shared list of available, empty houses.  This cycle repeats everyday.

## Key simulation variables

- number of households: 7,393
- fraction of household that are empty: 0.1
- desired neighborhood similarity fraction: 0.5

In [ ]:
schelling_config = ModelConfig(
    synth_pop=SynthPop("US_2010.v5", ["Kewaunee_County_WI"]),
    start_date="2023-01-01",
    end_date="2023-01-14",
)

schelling_job = Job(
    "model/schelling_block.fred",
    config=[schelling_config],
    key="cl_schelling_job",
    fred_version="11.0.1",
    results_dir="/home/epx/cl-results"
)

schelling_job.execute()

# the following loop idles while we wait for the simulation job to finish and periodically prints an update
update_count = 0
update_interval = 3
start_time = time.time()
timeout   = 300 # timeout in seconds
idle_time = 20   # time to wait (in seconds) before checking status again
while str(schelling_job.status) != 'DONE':
    if str(schelling_job.status) == 'ERROR':
        logs = schelling_job.status.logs
        log_msg = "; ".join(logs.loc[logs.level == "ERROR"].message.tolist())
        print(f"Job failed with the following error:\n '{log_msg}'")
        break
    if time.time() > start_time + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    
    if update_count >= update_interval:
        update_count = 0
        print(f"Job is still processing after {time.time() - start_time:.0f} seconds")
        
    update_count += 1
    
    time.sleep(idle_time)

print(f"Job completed in {time.time() - start_time:.0f} seconds")

str(schelling_job.status)

In [ ]:
# prep the data for visualization

block_history = schelling_job.results.csv_output("block_color_history.csv")
house_info = schelling_job.results.csv_output("household_info.csv")
agent_info = schelling_job.results.csv_output("agent_info.csv")
agent_house_history = schelling_job.results.csv_output("agent_house_history.csv")
agent_house_history = pd.merge(
    agent_house_history,
    house_info[["household_id","house_lat","house_long"]],
    how="left",on="household_id"
)

In the map below, each colored dot represents a householder agent who is either blue or red. Over time, the dots move around as agents who are unhappy seek out a new home in a block group that meets the similarity value defined the variable `desired_similarity`. The blue lines on the map show the boundaries of the block groups that are included in this simulation (these boundaries were downloaded from the [U.S. census website](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.2010.html#list-tab-OZSS213R7R9GVKFO5Y).)

This animated map shows that by about 14 days into the simulation, the population has reached a stable equilibrium situation characterized by completely homogenous block groups - all despite the fact that the population started with a roughly 50:50 spatial distribution of color and that people are happy living with up to 50% different colored agents.

In [ ]:
# calculate lat, long to pass to mapbox for map center
lat_cen = house_info['house_lat'].median()
long_cen = house_info['house_long'].median()

# set up Epistemix house map tiles
mapstyle="mapbox://styles/epxadmin/cm0ve9m13000501nq8q1zdf5p"
token="pk.eyJ1IjoiZXB4YWRtaW4iLCJhIjoiY20wcmV1azZ6MDhvcTJwcTY2YXpscWsxMSJ9._ROunfMS6hgVh1LPQZ4NGg"

# load the block group boundary file
f = open("Kewaunee_County_WI_blockgroups.geojson")
blocks_json = json.load(f)

# create the figure
fig1 = px.scatter_mapbox(
    agent_house_history, lat="house_lat", lon="house_long",
    color="agent_color",
    color_continuous_scale=[
                (0, "rgba(40,  95, 223, 1)"), # blue
                (1, "rgba(235, 90, 54,  1)")  # red
    ],
    animation_frame='date', height=700, zoom=9.25,
    labels={'agent_color':'Agent color'}
)

fig1.update_layout(
    mapbox={'layers': [{
        'source': blocks_json,
        'type': "line", 'below': "traces", 'color': "#6383F3",
        'opacity': 1.0
    }]}
)

fig1.update_layout(mapbox_style=mapstyle, mapbox_accesstoken=token)
fig1.update(layout_coloraxis_showscale=False)
fig1.update_layout(margin={"r":20,"t":10,"l":10,"b":3})
fig1.show()

We can also examine the aggregate characteristics of the county's block groups. The map below shows each block group of Kewaunee County, with the color representing the fraction of households that are blue.

In [ ]:
# calculate lat, long to pass to mapbox for map center
lat_cen = house_info['house_lat'].median()
long_cen = house_info['house_long'].median()

# set up Epistemix house map tiles
mapstyle="mapbox://styles/epxadmin/cm0ve9m13000501nq8q1zdf5p"
token="pk.eyJ1IjoiZXB4YWRtaW4iLCJhIjoiY20wcmV1azZ6MDhvcTJwcTY2YXpscWsxMSJ9._ROunfMS6hgVh1LPQZ4NGg"

# load the block group boundary file
f = open("Kewaunee_County_WI_blockgroups.geojson")
blocks_json = json.load(f)

# create the figure
fig2 = px.choropleth_mapbox(
    block_history, geojson=blocks_json,
    locations='blockgroup_id', color='frac_blue',
    color_continuous_scale=[
        (0,"rgba(235,90,54,1)"), # blue
        (1,"rgba(40, 95, 223, 1)") # red
        #(0,"rgba(249, 183, 45,1)"),
        #(1,"rgba(196, 200, 204,1)"),
        #(1,"rgba(104, 148, 255,1)")
    ],
    range_color=(0, 1),featureidkey="properties.GEOID10",
    hover_data=block_history.columns,
    animation_frame='date',
    zoom=9.25, height=700,
    center = {"lat": lat_cen,"lon": long_cen},
    labels={'frac_blue':'Blue fraction'}
)

fig2.update_layout(mapbox_style=mapstyle, mapbox_accesstoken=token)
fig2.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 300
fig2.update_layout(coloraxis_colorbar_orientation='h')
fig2.update_layout(coloraxis_colorbar_y=-0.1)
fig2.update_layout(margin={"r":0,"t":0,"l":0,"b":0.3})
fig2.show()

In [ ]:
# deleting our job now that we are done with it
schelling_job.delete(interactive=False)